<a href="https://colab.research.google.com/github/ST10059601/Ice-task-2-PDAN/blob/main/ice_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle --quiet

import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

from sklearn.metrics import classification_report, confusion_matrix

from google.colab import files
files.upload()

!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d puneet6060/intel-image-classification
!unzip -q intel-image-classification.zip -d intel_dataset

train_dir = "intel_dataset/seg_train/seg_train"
test_dir = "intel_dataset/seg_test/seg_test"

img_size = (150, 150)
batch_size = 32

# Training generator with validation split
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode="categorical",
    subset="training"
)

val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode="categorical",
    subset="validation"
)

# Test generator
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=False
)

class_labels = list(test_generator.class_indices.keys())


underfit_model = models.Sequential([
    layers.Conv2D(16, (3,3), activation='relu', input_shape=(150,150,3)),
    layers.MaxPooling2D(2,2),
    layers.Flatten(),
    layers.Dense(32, activation='relu'),
    layers.Dense(train_generator.num_classes, activation='softmax')
])

underfit_model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

history_underfit = underfit_model.fit(
    train_generator, epochs=3, validation_data=val_generator, verbose=1
)


overfit_model = models.Sequential([
    layers.Conv2D(64, (3,3), activation='relu', input_shape=(150,150,3)),
    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(256, (3,3), activation='relu'),
    layers.Conv2D(512, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Flatten(),
    layers.Dense(1024, activation='relu'),
    layers.Dense(train_generator.num_classes, activation='softmax')
])

overfit_model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

history_overfit = overfit_model.fit(
    train_generator, epochs=20, validation_data=val_generator, verbose=1
)

final_model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(150,150,3)),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(train_generator.num_classes, activation='softmax')
])

final_model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

callbacks = [
    EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True),
    ModelCheckpoint("best_model.h5", save_best_only=True)
]

history_final = final_model.fit(
    train_generator, epochs=10, validation_data=val_generator,
    callbacks=callbacks, verbose=1
)

y_pred = final_model.predict(test_generator)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = test_generator.classes

# Classification Report
print("Classification Report:\n")
print(classification_report(y_true, y_pred_classes, target_names=class_labels))

# Confusion Matrix
cm = confusion_matrix(y_true, y_pred_classes)
plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_labels, yticklabels=class_labels)
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix")
plt.show()


def plot_history(history, title):
    plt.figure(figsize=(12,5))
    plt.subplot(1,2,1)
    plt.plot(history.history['accuracy'], label='train')
    plt.plot(history.history['val_accuracy'], label='val')
    plt.title(f'{title} - Accuracy')
    plt.legend()

    plt.subplot(1,2,2)
    plt.plot(history.history['loss'], label='train')
    plt.plot(history.history['val_loss'], label='val')
    plt.title(f'{title} - Loss')
    plt.legend()
    plt.show()

plot_history(history_underfit, "Underfitting Model")
plot_history(history_overfit, "Overfitting Model")
plot_history(history_final, "Final Model")

misclassified_idx = np.where(y_pred_classes != y_true)[0]

plt.figure(figsize=(12, 8))
for i, idx in enumerate(misclassified_idx[:5]):
    img, label = test_generator[idx]
    plt.subplot(1, 5, i+1)
    plt.imshow(img[0])
    plt.title(f"T:{class_labels[y_true[idx]]}\nP:{class_labels[y_pred_classes[idx]]}")
    plt.axis("off")
plt.show()
